# Analysis of Data Job Market in Latvia and Lithuania
## Data Collection and preparation

### 0.Setting up Virtual Environment

In [1]:
!pip install virtualenv
!virtualenv myenv

created virtual environment CPython3.9.7.final.0-64 in 879ms
  creator CPython3Windows(dest=C:\Users\ernes\OneDrive\Mokslai\SGT- Final Project\Data_Job_Market_Analysis_LV\myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\ernes\AppData\Local\pypa\virtualenv)
    added seed packages: pip==23.1.2, setuptools==67.7.2, wheel==0.40.0
  activators BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
!myenv\Scripts\activate.bat

### 1. Data Collection

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
import re

### Latvian cv.lv

In [6]:
url = 'https://www.cv.lv/lv/search?limit=2000&offset=20&keywords%5B0%5D=data%20analyst&keywords%5B1%5D=data%20scientist&keywords%5B2%5D=data%20engineer&keywords%5B3%5D=machine%20learning&keywords%5B4%5D=analyst&keywords%5B5%5D=anal%C4%ABti%C4%B7is&keywords%5B6%5D=datu&keywords%5B7%5D=datu%20zin%C4%81tnieks&keywords%5B8%5D=big%20data%20analytics&sorting=EXPIRING&fuzzy=false&suitableForRefugees=false&isHourlySalary=false&isRemoteWork=false&isQuickApply=false'
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
vacancy_items = soup.find_all("li", class_="jsx-1871295890 jsx-2661613696 vacancies-list__item false")

In [9]:
vacancies = []
for item in vacancy_items:
    title = item.find("span", class_="jsx-1401030249 vacancy-item__title").text.strip()
    vacancy_item = item.find('a', {'class': 'vacancy-item'})
    vacancy_url = vacancy_item.get('href')
    end_date = item.find("span", class_="jsx-1401030249 vacancy-item__expiry").text.strip()
                                
    location = item.find("span", class_="jsx-1401030249 vacancy-item__locations").text.strip()
    salary = item.find("span", class_="jsx-1401030249 vacancy-item__salary-label")
    if salary is not None:
        salary = salary.text.strip()
    
    info_secondary = item.find('div', {'class': 'vacancy-item__info-secondary'})
    info_secondary_text = info_secondary.get_text()
    
    time_patterns = '(dien(a|s)|stund(a|s)|mēne(ši|sis)|sekund(e|es)|minūt(es|e))'
    pattern = f'Publicēts pirms (\d+) {time_patterns}|Atjaunināts pirms (\d+) {time_patterns}'
    match = re.search(pattern, info_secondary_text)
    published = match.group(0)
    
    p=1
    for a_tag in item.find_all('a', {'class': 'jsx-1401030249'}):
        p=p+1
        if p==3:
            company_name = a_tag.text
            company_url = a_tag['href']
        
      
    vacancies.append({
        "Title": title,
        "Vacancy URL" : vacancy_url,
        "Company name": company_name,
        "Company URL" :company_url,
        "Location": location,
        "Published": published,
        "End_date" : end_date, 
        "Salary": salary
    })

df = pd.DataFrame(vacancies)

### Lithuanian cv.lt

In [10]:
vacancies_lt = []

for page in range(1, 10): # Change the range according to the number of pages you want to scrape
    url_lt = f"https://www.cv.lt/darbo-paieska/data?page={page}&texts=data"
    response = requests.get(url_lt)
    soup = BeautifulSoup(response.content, "html.parser")
    vacancy_items = soup.select(".job-wr")

    for item in vacancy_items:
        title = item.find("button", class_="title").text.strip()
        vacancy_item= soup.select_one('.job-wr')
        vacancy_url = vacancy_item['href']
        company_name = item.find("span", {"class": "company"}).find("button").text.strip()

        # for Lithuanian website, company url is not available, adding for merging purposes
        company_url = "NaN"

        location = item.find("span", {"class": "company"}).find("span").text.strip()
        published = item.select_one(".time span").text

        # for Lithuanian website, expiry date is not available, adding for merging purposes
        end_date = "NaN"

        salary = item.find("span", {"class": "salary"})
        if salary is not None:
            salary = salary.text.strip()
        else:
            salary='NaN'

        vacancies_lt.append({
                "Title": title,
                "Vacancy URL": vacancy_url,
                "Company name": company_name,
                "Company URL": company_url,
                "Location": location,
                "Published": published,
                "End_date": end_date,
                "Salary": salary,
            })

df_lt = pd.DataFrame(vacancies_lt)

### 3. Data Cleaning

### cv.lv

In [11]:
cv_lv_link = r'https://www.cv.lv'
df['Vacancy URL'] = cv_lv_link + df['Vacancy URL']
df['Company URL'] = cv_lv_link + df['Company URL']

In [12]:
df['End_date'] = df['End_date'].str.replace('Beidzas: ', '')

In [13]:
df['Location'] = df['Location'].str.replace('—', '')
# split Location column into City, Region, Country
splitting = lambda x: pd.Series([i for i in reversed(x.split(','))])
df[['Country', 'Region', 'City']] = df['Location'].apply(splitting)

# Remove the original Location column
df.drop('Location', axis=1, inplace=True)

In [14]:
df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Salary,Country,Region,City
0,Izglītības departamenta direktora vietnieks iz...,https://www.cv.lv/lv/vacancy/998347/lr-izgliti...,LR Izglītības un zinātnes ministrija,https://www.cv.lv/lv/search/employer/lr-izglit...,Publicēts pirms 5 diena,17.05.2023,€ 2425 – 2695,Latvija,Rīgas rajons,Rīga
1,Junior Project Manager for eCommerce projects,https://www.cv.lv/lv/vacancy/986154/scandiweb/...,scandiweb,https://www.cv.lv/lv/search/employer/scandiweb...,Atjaunināts pirms 22 stunda,17.05.2023,€ 800 – 1350,Latvija,Rīgas rajons,Rīga
2,Mid-level CRO Strategist,https://www.cv.lv/lv/vacancy/986162/scandiweb/...,scandiweb,https://www.cv.lv/lv/search/employer/scandiweb...,Atjaunināts pirms 22 stunda,17.05.2023,€ 2000 – 3000,Latvija,Rīgas rajons,Rīga
3,VADOŠAIS IT ADMINISTRATORS/-E INFORMĀCIJAS TEH...,https://www.cv.lv/lv/vacancy/985999/bluor-bank...,BluOr Bank AS,https://www.cv.lv/lv/search/employer/bluor-ban...,Atjaunināts pirms 21 stunda,17.05.2023,€ 3500 – 4000,Latvija,Rīgas rajons,Rīga
4,LOĢISTIKAS SPECIĀLISTA PALĪGS,https://www.cv.lv/lv/vacancy/986401/konig-dist...,König Distribution AS,https://www.cv.lv/lv/search/employer/konig-dis...,Atjaunināts pirms 4 stunda,17.05.2023,€ 1300,Latvija,Rīgas rajons,Rīga
...,...,...,...,...,...,...,...,...,...,...
985,Test Engineer,https://www.cv.lv/lv/vacancy/993940/visma/test...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 13 diena,30.06.2023,€ 2000 – 3000,Latvija,Rīgas rajons,Rīga
986,Senior Test Engineer,https://www.cv.lv/lv/vacancy/995138/visma/seni...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 11 diena,30.06.2023,€ 2400 – 3000,Latvija,Rīgas rajons,Rīga
987,Security Manager,https://www.cv.lv/lv/vacancy/987624/visma/secu...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 27 diena,30.06.2023,€ 3500 – 4500,Latvija,Rīgas rajons,Rīga
988,Automatizācijas sistēmu inženieris/ Vadošais a...,https://www.cv.lv/lv/vacancy/992319/belam-riga...,Belam - Riga SIA,https://www.cv.lv/lv/search/employer/belam-rig...,Atjaunināts pirms 5 diena,27.07.2023,€ 2000,Latvija,Rīgas rajons,Rīga


In [15]:
# Remove the euro sign
df['Salary'] = df['Salary'].str.replace('€', '').str.strip()


# Check the format of each entry and apply the corresponding operation
df[['Salary Range min', 'Salary Range max']] = df['Salary'].str.split('–', n=1, expand=True)


In [16]:
df['Salary Range min'] = df['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
df['Salary Range max'] = df['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


C:\Users\ernes\AppData\Local\Temp/ipykernel_20412/1641069593.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Range min'] = df['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
C:\Users\ernes\AppData\Local\Temp/ipykernel_20412/1641069593.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Range max'] = df['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


In [17]:
for i in df.index:
    if pd.notna(df['Salary'][i]) and isinstance(df['Salary'][i], str) and '/st.' in df['Salary'][i]:
        if pd.notna(df['Salary Range min'][i]) and pd.notna(df['Salary Range max'][i]):
            df.loc[i, ['Salary Range min', 'Salary Range max']] *= 21.62 * 8


In [18]:
# Round the values to 2 decimal places
df['Salary Range min'] = df['Salary Range min'].round(2)
df['Salary Range max'] = df['Salary Range max'].round(2)

In [19]:
# remove the original Salary column
df.drop('Salary', axis=1, inplace=True)

In [20]:
import re
from datetime import datetime, timedelta

def get_date_from_string(s):
    if "dien" in s:
        days_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=days_ago)).strftime('%d.%m.%Y')
    elif "mēnesi" in s:
        months_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=months_ago*30)).strftime('%d.%m.%Y')
    elif "seku" in s or "min" in s or "stund" in s:
        return datetime.now().strftime('%d.%m.%Y')

# Apply the function to the column 'Published'
df['Published'] = df['Published'].apply(get_date_from_string)

### cv.lt

In [21]:
# Adding beginning for vacancy url to show full link
cv_lt_link = r'https://www.cv.lt'
df_lt['Vacancy URL'] = cv_lt_link + df_lt['Vacancy URL']

In [22]:
# split Location field, if more than one city- to show as 'Various'
df_lt['Location'] = df_lt['Location'].apply(lambda x: 'Various' if isinstance(x, str) and ',' in x else x)

In [23]:
# Replace Location field 'Kitas' to 'Other' 
df_lt['Location'] = df_lt['Location'].str.replace('Kitas', 'Other').str.strip()

In [24]:
# Adding columns to match LV formating for location

Country='Lithuania'
df_lt.insert(loc=df_lt.columns.get_loc('End_date') + 2, column='Country', value=Country)
df_lt.insert(loc=df_lt.columns.get_loc('End_date')+3,column='Region', value='Nan')
df_lt.insert(loc=df_lt.columns.get_loc('End_date')+4,column='City', value=df_lt['Location'])


# Remove the original Location column
df_lt.drop('Location', axis=1, inplace=True)

In [25]:
# Cleaning Salary field, if it is showing different value than salary amount, to remove and show it as 0
def clean_salary(salary):
    if salary.startswith("Pareiginės "):
        return 'NaN'
    else:
        return salary
    
df_lt["Salary"] = df_lt["Salary"].apply(clean_salary)

In [26]:
# Remove the euro sign 
df_lt['Salary'] = df_lt['Salary'].str.replace('€', '').str.strip()

# Remove the 'Nuo..' word
df_lt['Salary'] = df_lt['Salary'].str.replace('Nuo', '').str.strip()

# Remove the dot as a thousands separator
df_lt['Salary'] = df_lt['Salary'].str.replace('.', '').str.strip()

C:\Users\ernes\AppData\Local\Temp/ipykernel_20412/1120974158.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_lt['Salary'] = df_lt['Salary'].str.replace('.', '').str.strip()


In [27]:
# Spliting Salary field into two columns
df_lt[['Salary Range min', 'Salary Range max']] = df_lt['Salary'].str.extract(r'^(.*?)(?:-(.*))?$')

In [28]:
df_lt['Salary Range min'] = df_lt['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
df_lt['Salary Range max'] = df_lt['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)

C:\Users\ernes\AppData\Local\Temp/ipykernel_20412/35191948.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lt['Salary Range min'] = df_lt['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
C:\Users\ernes\AppData\Local\Temp/ipykernel_20412/35191948.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lt['Salary Range max'] = df_lt['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


In [29]:
# Round the values to 2 decimal places
df_lt['Salary Range min'] = df_lt['Salary Range min'].round(2)
df_lt['Salary Range max'] = df_lt['Salary Range max'].round(2)

In [30]:
# remove the original Salary column
df_lt.drop('Salary', axis=1, inplace=True)

In [31]:
# Formating job posting to show date instead of 'Before X days/months' etc.

import re
from datetime import datetime, timedelta

def get_date_from_string(s):
    if "d." in s:
        days_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=days_ago)).strftime('%d.%m.%Y')
    elif "mėn." in s:
        months_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=months_ago*30)).strftime('%d.%m.%Y')
    elif "val." in s or "min" in s:
        return datetime.now().strftime('%d.%m.%Y')

# Apply the function to the column 'Published'
df_lt['Published'] = df_lt['Published'].apply(get_date_from_string)

In [32]:
# Merging both dataframes into one
full_df = pd.concat([df, df_lt], ignore_index=True)

In [33]:
full_df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Country,Region,City,Salary Range min,Salary Range max
0,Izglītības departamenta direktora vietnieks iz...,https://www.cv.lv/lv/vacancy/998347/lr-izgliti...,LR Izglītības un zinātnes ministrija,https://www.cv.lv/lv/search/employer/lr-izglit...,11.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,2425.0,2695.0
1,Junior Project Manager for eCommerce projects,https://www.cv.lv/lv/vacancy/986154/scandiweb/...,scandiweb,https://www.cv.lv/lv/search/employer/scandiweb...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,800.0,1350.0
2,Mid-level CRO Strategist,https://www.cv.lv/lv/vacancy/986162/scandiweb/...,scandiweb,https://www.cv.lv/lv/search/employer/scandiweb...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,2000.0,3000.0
3,VADOŠAIS IT ADMINISTRATORS/-E INFORMĀCIJAS TEH...,https://www.cv.lv/lv/vacancy/985999/bluor-bank...,BluOr Bank AS,https://www.cv.lv/lv/search/employer/bluor-ban...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,3500.0,4000.0
4,LOĢISTIKAS SPECIĀLISTA PALĪGS,https://www.cv.lv/lv/vacancy/986401/konig-dist...,König Distribution AS,https://www.cv.lv/lv/search/employer/konig-dis...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,1300.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1165,Plungės miesto seniūnija Specialistas,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Plungės rajono savivaldybės administracija,NaN,11.05.2023,NaN,Lithuania,Nan,Plungė,NaN,NaN
1166,Turto valdymo skyriaus patarėjas,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Nacionalinė teismų administracija,NaN,10.05.2023,NaN,Lithuania,Nan,Vilnius,NaN,NaN
1167,Resocializacijos skyriaus atvejo vadybininkas ...,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Lietuvos kalėjimų tarnyba,NaN,09.05.2023,NaN,Lithuania,Nan,Vilnius,NaN,NaN
1168,Finansų politikos departamento Fiskalinės vald...,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Lietuvos Respublikos finansų ministerija,NaN,08.05.2023,NaN,Lithuania,Nan,Vilnius,NaN,NaN


### 4. Data Filtering

In [41]:
filtered_df = full_df[full_df['Title'].str.contains('anal|dat|duom', case=False)]

In [45]:
filtered_df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Country,Region,City,Salary Range min,Salary Range max
9,Middle+/Senior Business Analyst,https://www.cv.lv/lv/vacancy/986697/dataart-lt...,DataArt Ltd,https://www.cv.lv/lv/search/employer/dataart-l...,18.04.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,3500.0,4000.0
17,Partner Performance Analyst (for a fixed perio...,https://www.cv.lv/lv/vacancy/994377/tele2-sia/...,Tele2 SIA,https://www.cv.lv/lv/search/employer/tele2-sia...,10.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,2000.0,NaN
21,Junior Web Analytics,https://www.cv.lv/lv/vacancy/986160/scandiweb/...,scandiweb,https://www.cv.lv/lv/search/employer/scandiweb...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,1200.0,1800.0
23,ANALĪTIĶIS/-E Maksājumu pakalpojumu sniedzēju ...,https://www.cv.lv/lv/vacancy/986243/rietumu-ba...,Rietumu Banka AS,https://www.cv.lv/lv/search/employer/rietumu-b...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,1500.0,2200.0
27,Data Engineer,https://www.cv.lv/lv/vacancy/986233/epic-frog/...,Epic Frog,https://www.cv.lv/lv/search/employer/epic-frog...,16.05.2023,17.05.2023,Latvija,Rīgas rajons,Rīga,2845.0,4300.0
...,...,...,...,...,...,...,...,...,...,...,...
1104,Business Analyst in Capital Exposure Management,https://www.cv.lt/it-telekomunikaciju-darbai/i...,Danske Bank A/S Lietuvos filialas,NaN,08.05.2023,NaN,Lithuania,Nan,Vilnius,2680.0,4020.0
1105,Business Analyst in Capital Exposure Management,https://www.cv.lt/it-telekomunikaciju-darbai/i...,Danske Bank A/S Lietuvos filialas,NaN,05.05.2023,NaN,Lithuania,Nan,Vilnius,2680.0,4020.0
1152,Viešųjų finansų sistemos valdymo grupės finans...,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Lietuvos Respublikos finansų ministerija,NaN,11.05.2023,NaN,Lithuania,Nan,Vilnius,NaN,NaN
1161,Ekonominės analizės skyriaus vyriausiasis spec...,https://www.cv.lt/valstybes-tarnyba/duju-ir-el...,Valstybinė energetikos reguliavimo taryba,NaN,13.05.2023,NaN,Lithuania,Nan,Other,NaN,NaN


### 5. SQLite Database

In [47]:
import sqlite3
import pandas as pd

# create a connection and cursor to the database
conn = sqlite3.connect('Job_postings_CV.db')
cursor = conn.cursor()

# create all data table and later update it with new values 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='alldata'")
table_exists = cursor.fetchone()

if table_exists:
     # if 'alldata' table already exists, retrieve existing records to show what records already saved
    existing_records = pd.read_sql_query("SELECT * FROM alldata", conn)
    
    # add all data from filtered_df
    updated_df = pd.concat([existing_records, filtered_df])

    # drop duplicates based on all columns
    updated_df = updated_df.drop_duplicates()

    # update the 'alldata' table with the updated dataframe
    updated_df.to_sql('alldata', conn, if_exists='replace', index=False)
else:
    # if the 'alldata' table does not exist, create it
    filtered_df.to_sql('alldata', conn, index=False)

# close cursor and connection
cursor.close()
conn.close()